In [1]:
import numpy as np

In [2]:
# will return set of images of cardinality = num, each image has channels=3, width=3, height = 4
def get_set(num, w, h):
    set_ = []
    for i in xrange(num):
        image = []
        for j in xrange(3):
            channel = []
            for k in xrange(w):
                col = []
                for l in xrange(h):
                    col.append((i+1) * 1000 + (j+1) * 100 + (k+1) * 10 + (l+1))
                channel.append(col)
            channel = np.array(channel).transpose()
            image.append(channel)
        set_.append(image)
    set_=np.array(set_)
    return set_
set_ = get_set(8, 6, 6)
set_[0]

array([[[1111, 1121, 1131, 1141, 1151, 1161],
        [1112, 1122, 1132, 1142, 1152, 1162],
        [1113, 1123, 1133, 1143, 1153, 1163],
        [1114, 1124, 1134, 1144, 1154, 1164],
        [1115, 1125, 1135, 1145, 1155, 1165],
        [1116, 1126, 1136, 1146, 1156, 1166]],

       [[1211, 1221, 1231, 1241, 1251, 1261],
        [1212, 1222, 1232, 1242, 1252, 1262],
        [1213, 1223, 1233, 1243, 1253, 1263],
        [1214, 1224, 1234, 1244, 1254, 1264],
        [1215, 1225, 1235, 1245, 1255, 1265],
        [1216, 1226, 1236, 1246, 1256, 1266]],

       [[1311, 1321, 1331, 1341, 1351, 1361],
        [1312, 1322, 1332, 1342, 1352, 1362],
        [1313, 1323, 1333, 1343, 1353, 1363],
        [1314, 1324, 1334, 1344, 1354, 1364],
        [1315, 1325, 1335, 1345, 1355, 1365],
        [1316, 1326, 1336, 1346, 1356, 1366]]])

In [3]:
#mask shows how prepare convolution produce matrix of neighbourhood for one channel
mask = np.array([[00,00,00,00,11,12,00,21,22],
                 [00,00,00,12,13,14,22,23,24],
                 [00,00,00,14,15,16,24,25,26],
                 [00,21,22,00,31,32,00,41,42],
                 [22,23,24,32,33,34,42,43,44],
                 [24,25,26,34,35,36,44,45,46],
                 [00,41,42,00,51,52,00,61,62],
                 [42,43,44,52,53,54,62,63,64],
                 [44,45,46,54,55,56,64,65,66]])
#how channel look like
channel = [[11, 21, 31, 41, 51, 61],
           [12, 22, 32, 42, 52, 62],
           [13, 23, 33, 43, 53, 63],
           [14, 24, 34, 44, 54, 64],
           [15, 25, 35, 45, 55, 65],
           [16, 26, 36, 46, 56, 66]]
#result mask will count the occurances for each element of channel in prepared convolution matrix
result_mask = []
for row in channel:
    result_row = []
    for elem in row:
        result_row.append(np.sum((mask==elem).astype(np.int32)))
    result_mask.append(result_row)
result_mask = np.array(result_mask)
result_mask

array([[1, 2, 1, 2, 1, 1],
       [2, 4, 2, 4, 2, 2],
       [1, 2, 1, 2, 1, 1],
       [2, 4, 2, 4, 2, 2],
       [1, 2, 1, 2, 1, 1],
       [1, 2, 1, 2, 1, 1]])

In [6]:
#this block prepare result for backpropogation of prepare convolution same
set_result = []
for image in set_:
    image_result = []
    for channel in image:
        image_result.append(np.multiply(channel, result_mask))
    set_result.append(image_result)
set_result = np.array(set_result)

In [7]:
# block will create input matrix of size (3*6*6, num) where 3*6*6 is size of each image and num is number of images 

def convert_channel_to_column(channel):
    return np.ravel(channel, order='F')

def convert_image_to_column(image):
    result = []
    for channel in image:
        result += convert_channel_to_column(channel).tolist()
    return result

def convert_set_to_matrix(set_):
    result = []
    for image in set_:
        result += convert_image_to_column(image)
    return np.reshape(np.array(result), [3*6*6, set_.shape[0]], order='F')
                       
output_matrix = convert_set_to_matrix(set_result)

In [8]:
np.savetxt('/home/ahsan/squirrel_latest/squirrel/trunk/src/Test/test-case-data/prepare-conv-strided-same-back-r.txt', output_matrix, fmt='%i', delimiter=' ', header='108 8')